In [ ]:
# Alternative approach: Set PyTorch load behavior for compatibility
import torch
import warnings

# Temporarily suppress warnings
warnings.filterwarnings('ignore')

original_load = torch.load

def safe_torch_load(*args, **kwargs):
    kwargs.setdefault('weights_only', False)
    return original_load(*args, **kwargs)

torch.load = safe_torch_load

print("PyTorch load behavior updated for YOLO compatibility")

PyTorch load behavior updated for YOLO compatibility


In [ ]:
from comet_ml import start
from comet_ml.integration.pytorch import log_model

experiment = start(
    api_key="6yaczRR3RHvVzI5kL3KZt6DMy",
    project_name="gassy",
    workspace="furquanmobeen"
)

hyper_params = {
    "model_size": "yolov8m",
    "epochs": 100,
    "imgsz": 640,
    "batch_size": 16,
    "patience": 15,
    "learning_rate": 0.01,
    "weight_decay": 0.0005,
    "warmup_epochs": 3,
    "close_mosaic": 10,
    "optimizer": "AdamW",
    "device": "cuda",
    "amp": True,
    "cache": True
}

experiment.log_parameters(hyper_params)

print("Comet ML experiment initialized and hyperparameters logged")

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/furquanmobeen/gassy/b9fbe8e10c7e4256a91c1a516a166e60



Comet ML experiment initialized and hyperparameters logged


In [ ]:
from ultralytics import YOLO
import os
import torch
from pathlib import Path

model = YOLO('yolov8m.pt')

current_dir = Path.cwd()
dataset_path = current_dir / "gas tank yolo dataset.v3i.yolov8" / "data.yaml"

experiment.log_other("dataset_path", str(dataset_path))
experiment.log_other("model_type", "YOLOv8m")

results = model.train(
    data=str(dataset_path),
    epochs=100,
    imgsz=640,
    batch=16,
    patience=15,
    save=True,
    name='gas_tank_model',
    project='runs/detect',
    device="cuda",
    workers=0,
    plots=True,
    val=True,
    resume=False,
    amp=True,
    cache=True,
    optimizer='AdamW',
    lr0=0.01,
    weight_decay=0.0005,
    warmup_epochs=3,
    close_mosaic=10,
)

model_path = results.save_dir / "weights" / "best.pt"

best_model = YOLO(str(model_path))

log_model(experiment, model=best_model.model, model_name="gas_tank_yolov8m")

if hasattr(results, 'results_dict'):
    experiment.log_metrics(results.results_dict)

experiment.end()

New https://pypi.org/project/ultralytics/8.3.207 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.11.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=c:\Users\furqu\OneDrive\UCLL\Projects\Gassy\gass_GASSY\gas tank yolo dataset.v3i.yolov8\data.yaml, epochs=100, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=cuda, workers=0, project=runs/detect, name=gas_tank_model, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

Error logging model to Comet ML: 'pickle_module'
